In [3]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import IPython.display as ipd
from PIL import Image, ImageDraw, ImageFont
from sympy import Point, Polygon 
from tqdm.notebook import tqdm
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [4]:
def exercitiu_live(tip_exercitiu):
    #Pentru Vid live
    #Feed-ul de la camera
    global pose_model
    global puncte_linii_suport_dr
    global puncte_linii_suport_st
    global puncte
    global stare
    global cnt

    cnt = 0
    stare = 0
    puncte_linii_suport_dr = [mp_pose.PoseLandmark.RIGHT_FOOT_INDEX, mp_pose.PoseLandmark.RIGHT_HEEL, mp_pose.PoseLandmark.RIGHT_ANKLE, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_SHOULDER]
    puncte_linii_suport_st = [mp_pose.PoseLandmark.LEFT_FOOT_INDEX, mp_pose.PoseLandmark.LEFT_HEEL, mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_SHOULDER]
    cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 960)
    
    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
        while cam.isOpened():
            ret, frame = cam.read()
    
            #colorarea frame-ului din BGR in RGB
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img.flags.writeable = False
    
            #detectarea modelului
            pose_model = pose.process(img)
            try:
                puncte = pose_model.pose_landmarks.landmark
            except Exception as e:
                print(f"Eroare de detectie a modelului: {e}")
            
            #colorarea frame-ului din RGB in BGR
            img.flags.writeable = True
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
            mp_drawing.draw_landmarks(img, pose_model.pose_landmarks, mp_pose.POSE_CONNECTIONS)
           

            
            img = analiza_fandare_front(img)
                
            cv2.imshow("Camera Feed", img)
            
            cv2.waitKey(1)
            if cv2.getWindowProperty("Camera Feed", cv2.WND_PROP_VISIBLE) < 1:
                break
        cam.release()
        cv2.destroyAllWindows()

In [5]:
def adaug_text(img, indicatii):
    try:
        dif = 5
        for indicatie in indicatii:
            img = cv2.putText(img, indicatie, (5, img.shape[0] - dif), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            dif += 40
    except Exception as e:
        print(f"Eroare la adaug_text: {e}")
    return img

In [6]:
def analiza_fandare_front(img):

    coord_pct_suport_dr = []
    coord_pct_suport_st = []
    
    
    for (puncte_cheie_dr, puncte_cheie_st) in zip(puncte_linii_suport_dr, puncte_linii_suport_st) :
        if puncte[puncte_cheie_dr.value].visibility > 0.5 and  puncte[puncte_cheie_st.value].visibility > 0.5:
            coord_pct_suport_dr.append((int(puncte[puncte_cheie_dr.value].x * img.shape[1]), int(puncte[puncte_cheie_dr.value].y * img.shape[0]), int(puncte[puncte_cheie_dr.value].z)))
            coord_pct_suport_st.append((int(puncte[puncte_cheie_st.value].x * img.shape[1]), int(puncte[puncte_cheie_st.value].y * img.shape[0]), int(puncte[puncte_cheie_st.value].z)))

    #if stare == 0:
    indicatii = pregatire_fandare(coord_pct_suport_dr, coord_pct_suport_st)

    #if stare == 1:
        #indicatii = proces_fandare(coord_pct_suport_dr, coord_pct_suport_st)
    
    img = adaug_text(img, indicatii)
    

    
    
    return img

In [7]:
def proces_fandare(coord_pct_suport_dr, coord_pct_suport_st):
    #aici as putea introduce  calculul scorului cand fac exercitiul
    global cnt
    global stare
    indicatii = [f"Counter: {cnt}"]
    
    flexie_genunchi_dr = calcul_unghi_2_drepte(coord_pct_suport_dr[2], coord_pct_suport_dr[3], coord_pct_suport_dr[3], coord_pct_suport_dr[4])
    flexie_genunchi_st = calcul_unghi_2_drepte(coord_pct_suport_st[2], coord_pct_suport_st[3], coord_pct_suport_st[3], coord_pct_suport_st[4])

    if flexie_genunchi_dr > 60 and flexie_genunchi_st > 60:
        cnt += 1
        stare = 0

    return indicatii
    
    

In [8]:
def pregatire_fandare(coord_pct_suport_dr, coord_pct_suport_st):

    global stare
    indicatii = [f"Counter: {cnt}"]
    unghi_talpi = False
    pozitionare_picioare = False
    
    if len(coord_pct_suport_dr) == 6 and len(coord_pct_suport_st) == 6:
        
        unghi_talpa_dr = calcul_unghi_orizont(coord_pct_suport_dr[0], coord_pct_suport_dr[1])
        unghi_talpa_st = 180 - calcul_unghi_orizont(coord_pct_suport_st[0], coord_pct_suport_st[1])

        
        if  45 <=  unghi_talpa_dr <= 75 and  45 <=  unghi_talpa_st <= 75:
            unghi_talpi = True
        else:
            indicatii.append("unghiul talpilor incorect")
    
        if coord_pct_suport_dr[2][0] < coord_pct_suport_dr[4][0] and coord_pct_suport_st[2][0] > coord_pct_suport_st[4][0]:
            pozitionare_picioare = True
        else:
            indicatii.append("picioarele prea apropiate")
        
    
        if unghi_talpi and pozitionare_picioare:
            stare = 1
            indicatii.append("pozitie corecta!")
        print (stare)
        print(unghi_talpa_dr)
        print(unghi_talpa_st)
        print(abs(unghi_talpa_dr - unghi_talpa_st))

    else:
        indicatii.append("intrati in cadru")
    return indicatii

In [9]:
def calcul_unghi_orizont(a, b):
    a = np.array(a)
    b = np.array(b)
    c = [0, 0.5, 0]
    d = [0.5, 0.5, 0]
    vect_dir_ab = np.subtract(b, a)
    vect_dir_cd = np.subtract(d, c)
    
    lungime_ab = np.linalg.norm(vect_dir_ab)
    lungime_cd = np.linalg.norm(vect_dir_cd)
    unghi = np.arccos(np.dot(vect_dir_ab, vect_dir_cd) / (lungime_ab * lungime_cd))
    unghi = np.rad2deg(unghi)

    return unghi

In [10]:
def calcul_unghi_2_drepte(a, b, c, d):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    d = np.array(d)
    
    vect_dir_ab = np.subtract(b, a)
    vect_dir_cd = np.subtract(d, c)
    
    lungime_ab = np.linalg.norm(vect_dir_ab)
    lungime_cd = np.linalg.norm(vect_dir_cd)
    unghi = np.arccos(np.dot(vect_dir_ab, vect_dir_cd) / (lungime_ab * lungime_cd))
    unghi = np.rad2deg(unghi)

    return unghi

In [11]:
def afla_orientare(puncte):
    sold_stg = puncte[mp_pose.PoseLandmark.LEFT_HIP.value].z
    sold_drept = puncte[mp_pose.PoseLandmark.RIGHT_HIP.value].z

    if sold_stg > 0:
        return "stanga"
    return "dreapta"

In [13]:
exercitiu_live(1)

0
69.04422326936782
93.0940580589171
24.04983478954928
0
65.65891006330749
76.7594800848128
11.100570021505305
0
65.89776549883887
76.7594800848128
10.861714585973928


In [11]:
#Pentru salvare VID
#Feed-ul de la camera
vid_original = cv2.VideoCapture('video_test_4.mp4')
nr_frames = int(vid_original.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vid_original.get(cv2.CAP_PROP_FPS)

vid_prelucrat = cv2.VideoWriter('video_prelucrat.mp4',  cv2.VideoWriter_fourcc(*'MP4V'), fps, (int(vid_original.get(3)), int(vid_original.get(4)))) 

with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    for frame_index in tqdm(range(nr_frames)):
        ret, frame = vid_original.read()

        #colorarea frame-ului din BGR in RGB
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #img.flags.writeable = False
        #detectarea modelului
        pose_model = pose.process(img)
        
        #colorarea frame-ului din RGB in BGR
        img.flags.writeable = True
        #img = prelucrare_img(frame, 1)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(img, pose_model.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        vid_prelucrat.write(img)
            
    vid_original.release()
    vid_prelucrat.release()

  0%|          | 0/904 [00:00<?, ?it/s]